In [1]:
import random
import numpy as np

In [6]:
def MM1_with_reneging(arrival_rate, service_rate, patience_rate, num_customers):
    arrival_times = []
    start_service_times = []
    finish_service_times = []
    waiting_times = []
    
    current_time = 0
    num_served = 0
    num_reneged = 0

    for i in range(num_customers):
        # --- 1. 產生到達時間 ---
        inter_arrival = random.expovariate(arrival_rate)
        if i == 0:
            arrival_times.append(inter_arrival)
        else:
            arrival_times.append(arrival_times[i-1] + inter_arrival)

        arrival = arrival_times[i]

        # --- 2. 顧客的「耐心時間」 ---
        patience_time = random.expovariate(patience_rate)   # 平均耐心 1/patience_rate
        give_up_time = arrival + patience_time

        # 伺服器可以開始服務他的時間（依舊是 max(到達, 目前服務完時間)）
        potential_start = max(arrival, current_time)

        # --- 3. 判斷會不會撐到開始服務 ---
        if give_up_time <= potential_start:
            # 撐不到開始服務就走了 → reneging
            num_reneged += 1
            # current_time 不變，因為伺服器繼續做原來的事或是空著等下一個人
            continue

        # --- 4. 他願意等到開始服務，正常服務流程 ---
        start_service_time = potential_start
        service_time = random.expovariate(service_rate)
        finish_service_time = start_service_time + service_time

        start_service_times.append(start_service_time)
        finish_service_times.append(finish_service_time)
        waiting_times.append(start_service_time - arrival)

        current_time = finish_service_time
        num_served += 1

    # --- 統計指標 ---
    if num_served > 0:
        total_time = finish_service_times[-1] - arrival_times[0]
        busy_time  = sum(finish - start for start, finish in zip(start_service_times, finish_service_times))

        lambda_eff = num_served / total_time          # 有被服務的有效到達率
        avg_waiting_time = sum(waiting_times) / num_served
        avg_service_time = busy_time / num_served

        Wq = avg_waiting_time
        W  = Wq + avg_service_time
        Lq = lambda_eff * Wq
        L  = lambda_eff * W
    else:
        L = Lq = W = Wq = 0

    return L,Lq,W,Wq,num_served,num_reneged


## Simulation Setup

In [7]:
LAMBDA = 2  # Arrival rate
MU = 4      # Service rate
PATIENT = 1  # Patience rate
CUSTOMERS = 1000
RO = LAMBDA / MU  # Traffic intensity




In [8]:
L, Lq,W,Wq, served, reneged = MM1_with_reneging(LAMBDA, MU, PATIENT, CUSTOMERS)
print(f"Results for M/M/1 with Reneging (λ={LAMBDA}, μ={MU}, θ={PATIENT}, Customers={CUSTOMERS}):")
print(f"L  = {L}")
print(f"Lq = {Lq}")
print(f"W  = {W}")
print(f"Wq = {Wq}")
print(f"Number of served customers: {served}")
print(f"Number of reneged customers: {reneged}")

Results for M/M/1 with Reneging (λ=2, μ=4, θ=1, Customers=1000):
L  = 0.609909362566637
Lq = 0.17458775687876613
W  = 0.3501596315082368
Wq = 0.10023388451892981
Number of served customers: 873
Number of reneged customers: 127
